In [ ]:
import mlflow
#import optuna
import pickle
from pathlib import Path
import lightgbm as lgb
import os
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit

# Set directory for mlflow runs
base_dir = str(Path.cwd().parents[1])
MLFLOW_TRACKING_URI = base_dir + '/mlflow/load_forecasting_analysis/'
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

############################## Activate local web host ##############################
# mlflow server --host 127.0.0.1 --port 5000 --backend-store-uri MLFLOW_TRACKING_URI
#####################################################################################

# Multiple outputs per notebook cell
%config InteractiveShell.ast_node_interactivity = 'all'


In [13]:
os.environ["MLFLOW_TRACKING_URI"]

'c:\\Users\\WulfN/mlflow/load_forecasting_analysis/'

Load Data

In [8]:
cd = Path.cwd()
data_dir = str(cd.parents[1])
upsampled = data_dir + '/datasets/country_energy/load_wthr_downsample.pickle'
downsampled = data_dir + '/datasets/country_energy/load_wthr_upsample.pickle'

# Xu - upsampled
with open(upsampled, 'rb') as f:
    Xu = pickle.load(f)

# Xd - downsampled
with open(downsampled, 'rb') as f:
    Xd = pickle.load(f)

Prepare Data for Model Training

In [ ]:
X = Xd.drop(columns='load_actual') # set to x
y = Xd['load_actual']

ts_cv = TimeSeriesSplit(n_splits=10)

    for i, (train_index, test_index) in enumerate(ts_cv.split(X)):
        print(f"Fold {i}:")
        print(f"  Train: index={train_index}")
        print(f"  Test:  index={test_index}")

Objective Function (LightGBM)

In [ ]:
def objective(trial, x, y): # n_splits=10, # adjust
    
    # Prelim Params
    # params = {
    #     'n_estimators': trial.suggest_int('n_estimators', 100, 1000), # adjust
    #     'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5),
    #     'num_leaves': trial.suggest_int('num_leaves', 31, 511), # adjust
    #     'max_depth': trial.suggest_int('max_depth', 3, 9), # adjust if overfit
    #     'subsample': trial.suggest_float('subsample', 0.5, 1), # research
    #     'colsample_bytree': trial.suggest-float('colsample_bytree', 0.7, 1),
    #     'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
    #     'reg_lambda': trial.suggest_float('reg_lambda', 0, 1)
    # }

    # simplified params for testing
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1)
    }

    # Consider specifying a param for the number of splits
    ts_cv = TimeSeriesSplit(n_splits=10)

    for i, (train_index, test_index) in enumerate(ts_cv.split(X)):
        print(f"Fold {i}:")
        print(f"  Train: index={train_index}")
        print(f"  Test:  index={test_index}")

    # Example splits data and fits models on the different folds
    # then stores resulting mae and mape

    # Understand timeseriessplit

    # Cross val - Is this done by repeatedly calling objective function?

    # Parallel processing (if possible) - This may be in the model training section
    # https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/004_distributed.html

    # Store results metrics (mape, mae)

    # Average across cross validated datasets

Model Training and Hyperparameter Selection

In [ ]:
# Set random string
    # ensure if covers mlfflow and other processes

# parallelization with mlflow
# https://mlflow.org/docs/latest/traditional-ml/hyperparameter-tuning-with-child-runs/notebooks/index.html

# study.trials_dataframe()


Results Graphs (MLflow)

In [ ]:
# Can optuna or MLflow assist in this
# mlflow.set_experiment("check-localhost-connection")
# mlflow.set_experiment("LightGBM Forecasting")

# with mlflow.start_run():
#     mlflow.log_metric("foo", 1)
#     mlflow.log_metric("bar", 2)


2025/02/20 13:53:40 INFO mlflow.tracking.fluent: Experiment with name 'check-localhost-connection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/140043457852554855', creation_time=1740084820943, experiment_id='140043457852554855', last_update_time=1740084820943, lifecycle_stage='active', name='check-localhost-connection', tags={}>

🏃 View run omniscient-cod-280 at: http://127.0.0.1:5000/#/experiments/140043457852554855/runs/b337cbc6f61c4ea7960c2d7baf56f10e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/140043457852554855


Feature Importance of Best Model, MAPE, MAE

Best Model on final validation set